In [74]:
import dataprocess as dp
import transformers
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ExponentialLR
from modelhelper import MSE_Vec_matrix
import torch
from torchvision import models

#TODO Normalize Xtest
#Make our own loss function

# Hyperparameters
# Batch Size, Num Epochs, Learning Rate, Momentum, FC Layer, Activation Function


# 0 Lowercase & N2W
# 1 Contractions
# 2 Remove Punctutations
# 3 Strop Words

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def accuracy(net, input_id, labels):
    j = len(labels)

    guesses = torch.argsort(net(input_id), dim=1, descending=True)
#     print("GUESS: ", guesses[:,0])

    # guess1 = torch.argmax(guesses, dim=1)
    # guess2 = torch.argsort(guesses, dim=1)[-2]


    # current_real = torch.tensor(labels)
    current_real = labels.clone().detach()
#     print("LABEL: ", current_real)

    top1_acc = torch.sum(current_real==guesses[:,0])
    top2_acc = torch.sum(current_real==guesses[:,1])


    # print(guesses[:100])
    # print(current_real[:100])
    # print("Guess: ", guesses, "Label: ", current_real)

    accuracy_1 = top1_acc / j
    accuracy_2 = (top2_acc + top1_acc) / j
    # print(running_acc)
    # print(j)

    # print('Accuracy for top 1: %d %%' % ((accuracy_1) * 100.0))
    # print('Accuracy for top 2: %d %%' % ((accuracy_2) * 100.0))

    return accuracy_1.item(), accuracy_2.item()

    

In [2]:
# !cd GitHub/EC523_Project/
# !pip install transformers

In [2]:
data = dp.unpickle_data("reviews_Electronics_5_7_encoded.pickle")

In [23]:
sentences = data['reviewText']
labels = data['overall']
print(sentences.shape)
print(len(labels))

torch.Size([1689188, 768])
1689188


In [24]:
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.1, random_state=42, stratify=labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

X_test = X_test.clone().detach()
y_test = torch.tensor(y_test).cuda()

for i in range(len(y_test)):
    y_test[i] = y_test[i] - 1 


# indecies = torch.permute(indecies)
# X_train = X_train[indexes,:]

X_train = X_train.clone().detach()
y_train = torch.tensor(y_train)

In [25]:
rev1 = X_train[y_train==1]
rev2 = X_train[y_train==2]
rev3 = X_train[y_train==3]
rev4 = X_train[y_train==4]
rev5 = X_train[y_train==5]

training_points = min(len(rev1), len(rev2), len(rev3), len(rev4), len(rev5))

rev1 = rev1 [:training_points]
rev2 = rev2 [:training_points]
rev3 = rev3 [:training_points]
rev4 = rev4 [:training_points]
rev5 = rev5 [:training_points]

indicies = torch.randperm(training_points * 5)

X_train = torch.cat((rev1, rev2, rev3, rev4, rev5), dim=0)[indicies, :]
base = torch.zeros(training_points)
y_train = torch.cat((base,base+1,base+2,base+3,base+4))[indicies]


In [36]:
class Net(nn.Module):
    def __init__(self, h_sizes, dropout=0.5, activation=F.sigmoid):

        # self.loss = torch.nn.MSELoss()
        super(Net, self).__init__()
        self.activation = activation
        self.h_sizes = h_sizes
        self.num_layers = len(self.h_sizes)
        self.fc = nn.ModuleList()
        self.dropout = nn.Dropout(dropout)

        for i in range(self.num_layers):
            self.fc.append(nn.Linear(self.h_sizes[i][0], self.h_sizes[i][1]))


    def forward(self, x):
        activation = self.activation
        for i in range(self.num_layers):
            if i == self.num_layers - 1:
                x = F.softmax(self.fc[i](x), dim=1)
            else:
                x = self.dropout(activation(self.fc[i](x)))
        return x

The cell below handles all the MLP Variables

In [66]:
# Learning Rate Decay
LR_START  = 1e-1
LR_END    = 1e-4
LR_GAMMA  = (LR_END/LR_START)**(1/NUM_EPOCH)

NUM_EPOCH = 10000
dropout = 0
hidden_layers = [[768, 500], [500, 250], [250, 5]]
activation_func = F.sigmoid

net = Net(  h_sizes=hidden_layers,
            dropout=dropout, 
            activation=activation_func).to(device)

print(net)

optimizer = torch.optim.SGD(net.parameters(), lr=LR_START, momentum=0.9)
criterion = torch.nn.MSELoss().to(device)
scheduler = ExponentialLR(optimizer, gamma=LR_GAMMA)
MSE_vec   = MSE_Vec_matrix(mid=1)
MSE_vec.to(device)

Net(
  (fc): ModuleList(
    (0): Linear(in_features=768, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=250, bias=True)
    (2): Linear(in_features=250, out_features=5, bias=True)
  )
  (dropout): Dropout(p=0, inplace=False)
)


In [62]:
X_train = X_train.to(device)
y_train = y_train.long().to(device)
X_test  = X_test.to(device)

$$
\gamma = \left( \frac{\alpha}{\beta} \right)^{\frac{1}{e}}
$$

gamma is decay rate

beta is start learning rate

alpha is ending learning rate

e is number of epochs

In [79]:
batch_size = 250
indecies = torch.tensor(range(batch_size))

train_losses = []
test_losses = []
accuracy1 = []
accuracy2 = []
for epoch in range(NUM_EPOCH):
    running_loss = 0.0
    for i in range(len(y_train)//batch_size):
    #for i in range(1):
        select = batch_size * i + indecies
        optimizer.zero_grad()
        outputs = net(X_train[select,:])

        # MSE Loss
        loss = criterion(outputs, MSE_vec[y_train[select]])

        # Cross Entropy Loss
#         loss = criterion(outputs, y_train[select])

        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    with torch.no_grad():
        training_loss = criterion(net(X_test),MSE_vec[y_test.long()]).item()
        # Added .eval() to account for the added dropout layers
        ac1,ac2 = accuracy(net.eval(), X_test, y_test)
        accuracy1.append(ac1)
        accuracy2.append(ac2)
    
    print('[%d] loss: %.3f \t test loss: %.3f \t test_ac1: %.2f \t test_ac2: %.2f' %
    (epoch + 1, running_loss, training_loss*10,ac1,ac2))
    scheduler.step()
    
train_losses.append(running_loss)
test_losses.append(training_loss)
running_loss = 0.0

  

print('Finished Training')
plt.plot(train_losses)
plt.figure()
plt.plot(test_losses)
plt.figure()
plt.plot(accuracy1)
plt.figure()
plt.plot(accuracy2)


[1] loss: 201.457 	 test loss: 1.280 	 test_ac1: 0.47 	 test_ac2: 0.70
[2] loss: 201.487 	 test loss: 1.295 	 test_ac1: 0.46 	 test_ac2: 0.69


KeyboardInterrupt: 

In [78]:
net = net.eval()

train_acc = accuracy(net, X_train.cuda(), y_train.cuda())
test_acc = accuracy(net, X_test, y_test)

TypeError: type tuple doesn't define __round__ method